In [1]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

from torch.utils.data import DataLoader
from datasets import Dataset, load_dataset
from evaluate import load
import numpy as np

from typing import List, Dict


2025-02-19 11:26:10.634331: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-19 11:26:10.643891: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739944570.654913    8815 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739944570.658395    8815 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-19 11:26:10.670146: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
import dataloading
import model_define

2025-02-19 11:26:56.316751: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
pl.seed_everything(445326, workers=True)

Seed set to 445326


445326

In [6]:
data = dataloading.YelpDataLoader()
data.prepare_data()

In [7]:
data.setup()
print(len(data.train))
print(len(data.val))
print(len(data.test))

Parameter 'function'=<function encoder_factory.<locals>.encode at 0x72c21e9e4ea0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100%|██████████| 760/760 [00:37<00:00, 20.20 examples/s]

11200
5600
760


In [31]:
from model_define import Model
model = Model()

In [35]:
MAX_EPOCHS = 15

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor="val_loss",
    dirpath="model",
    filename="yelp-sentiment-multilingual-{epoch:02d}-{val_loss:.3f}",
    save_top_k=3,
    mode="min")

trainer = pl.Trainer(max_epochs=MAX_EPOCHS, 
                     callbacks=[checkpoint_callback])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [36]:
trainer.fit(model, data.train_dataloader(), data.val_dataloader())


  | Name   | Type       | Params | Mode 
----------------------------------------------
0 | layers | Sequential | 492 K  | train
----------------------------------------------
492 K     Trainable params
0         Non-trainable params
492 K     Total params
1.971     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


/home/skullboxml/minor_project/venv/lib/python3.12/site-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Epoch 14: 100%|██████████| 350/350 [00:11<00:00, 29.47it/s, v_num=0, train_loss_step=0.310, train_accuracy_step=0.875, val_loss_step=0.138, val_accuracy_step=0.938, val_loss_epoch=0.203, val_accuracy_epoch=0.921, train_loss_epoch=0.296, train_accuracy_epoch=0.876] 

`Trainer.fit` stopped: `max_epochs=15` reached.


Epoch 14: 100%|██████████| 350/350 [00:11<00:00, 29.46it/s, v_num=0, train_loss_step=0.310, train_accuracy_step=0.875, val_loss_step=0.138, val_accuracy_step=0.938, val_loss_epoch=0.203, val_accuracy_epoch=0.921, train_loss_epoch=0.296, train_accuracy_epoch=0.876]


In [37]:
trainer.test(dataloaders=data.test_dataloader())

/home/skullboxml/minor_project/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at /home/skullboxml/minor_project/model/yelp-sentiment-multilingual-epoch=07-val_loss=0.196.ckpt
Loaded model weights from the checkpoint at /home/skullboxml/minor_project/model/yelp-sentiment-multilingual-epoch=07-val_loss=0.196.ckpt


Testing DataLoader 0: 100%|██████████| 24/24 [00:00<00:00, 50.10it/s]


/home/skullboxml/minor_project/venv/lib/python3.12/site-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 24. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9013158082962036     │
│         test_loss         │    0.25619542598724365    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.25619542598724365, 'test_accuracy': 0.9013158082962036}]

In [40]:
best_model = Model.load_from_checkpoint(checkpoint_callback.best_model_path)

In [41]:
from pprint import PrettyPrinter
pp = PrettyPrinter()

def predict(text: List[str]):
    embeddings = torch.Tensor(dataloading.embed_text(text))
    logits = best_model(embeddings)
    preds = torch.argmax(logits, dim=1).detach().cpu().numpy()
    scores = torch.softmax(logits, dim=1).detach().cpu().numpy()

    results = []
    for t, best_index, score_pair in zip(text, preds, scores):
        results.append({
            "text": t,
            "label": "positive" if best_index == 1 else "negative",
            "score": score_pair[best_index]
        })
    return results

In [42]:
english_text = "Like any Barnes & Noble, it has a nice comfy cafe, and a large selection of books. The staff is very friendly and helpful. They stock a decent selection, and the prices are pretty reasonable."

german_translation = "Wie jedes Barnes & Noble hat es ein nettes, gemütliches Café und eine große Auswahl an Büchern. Das Personal ist sehr freundlich und hilfsbereit. Sie haben eine anständige Auswahl und die Preise sind ziemlich vernünftig."

pp.pprint(predict([english_text, german_translation]))

[{'label': 'positive',
  'score': np.float32(0.73088825),
  'text': 'Like any Barnes & Noble, it has a nice comfy cafe, and a large '
          'selection of books. The staff is very friendly and helpful. They '
          'stock a decent selection, and the prices are pretty reasonable.'},
 {'label': 'positive',
  'score': np.float32(0.96647483),
  'text': 'Wie jedes Barnes & Noble hat es ein nettes, gemütliches Café und '
          'eine große Auswahl an Büchern. Das Personal ist sehr freundlich und '
          'hilfsbereit. Sie haben eine anständige Auswahl und die Preise sind '
          'ziemlich vernünftig.'}]


/tmp/ipykernel_8815/957889146.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  embeddings = torch.Tensor(dataloading.embed_text(text))


In [43]:
english_text = "The inside of the Restaurant was not clean at all. And we also did not like their lighting arrangement. Too dark."

italian_translation = "L'interno del Ristorante non era affatto pulito. E non ci piaceva nemmeno la loro disposizione delle luci. Troppo scuro."

finnish_translation = "Ravintolan sisäpuoli ei ollut ollenkaan puhdas. Ja emme myöskään pitäneet heidän valaistusjärjestelystä. Liian pimeä."

pp.pprint(predict([english_text, italian_translation, finnish_translation]))

[{'label': 'negative',
  'score': np.float32(0.98972845),
  'text': 'The inside of the Restaurant was not clean at all. And we also did '
          'not like their lighting arrangement. Too dark.'},
 {'label': 'negative',
  'score': np.float32(0.75943637),
  'text': "L'interno del Ristorante non era affatto pulito. E non ci piaceva "
          'nemmeno la loro disposizione delle luci. Troppo scuro.'},
 {'label': 'positive',
  'score': np.float32(0.90973556),
  'text': 'Ravintolan sisäpuoli ei ollut ollenkaan puhdas. Ja emme myöskään '
          'pitäneet heidän valaistusjärjestelystä. Liian pimeä.'}]


In [47]:
english_text = "The inside of the Restaurant was not clean at all. And we also did not like their lighting arrangement. Too dark."

telugu_translation = "restaurant parishubramga ledu mariyu aa deepalu nachaledu"

finnish_translation = "Ravintolan sisäpuoli ei ollut ollenkaan puhdas. Ja emme myöskään pitäneet heidän valaistusjärjestelystä. Liian pimeä."

pp.pprint(predict([english_text, telugu_translation, finnish_translation]))

[{'label': 'negative',
  'score': np.float32(0.93024975),
  'text': 'The inside of the Restaurant was not clean at all. And we also did '
          'not like their lighting arrangement. Too dark.'},
 {'label': 'negative',
  'score': np.float32(0.87560916),
  'text': 'restaurant parishubramga ledu mariyu aa deepalu nachaledu'},
 {'label': 'positive',
  'score': np.float32(0.9815802),
  'text': 'Ravintolan sisäpuoli ei ollut ollenkaan puhdas. Ja emme myöskään '
          'pitäneet heidän valaistusjärjestelystä. Liian pimeä.'}]


In [59]:
import asyncio
from googletrans import Translator

async def translate_text():
    translator = Translator()
    original_text = "این غذا خیلی شوره!"
    translation = await translator.translate(original_text, dest="en")
    print(translation.text) 

await translate_text()


This food is too dandelion!


In [ ]:
from googletrans import Translator

translator = Translator()
text = input()

detected_lang = translator.detect(text)
if detected_lang == "en":
    pp.pprint(predict([text]))
else:
    translated_text = translate_text(text)
    pp.pprint(predict([translated_text]))